In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [12]:
cam = cv2.VideoCapture(0)

# Object handDetector
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)

detectionResults = {}

while True:
    # Read from camera
    success, frame = cam.read()

    # Pose Detection
    detectionResults['hands'], frame = handDetector.findHands(frame, draw=True)
    frame, detectionResults['face'] = faceDetector.findFaces(frame, draw=True)
    frame = poseDetector.findPose(frame, draw=True)
    detectionResults['pose_landmarks'], detectionResults['pose_bbox'] = poseDetector.findPosition(frame, bboxWithHands=False)

    # Hand is a dictionary of: 
    #   lmList: List of 21 landmark points
    #   bbox: Bounding Box info x,y,w,h
    #   center: Center coordinates cx,cy
    #   type: left or right

    # Show result
    cv2.imshow("Image", frame)
    keyPressed = cv2.waitKey(10)
    if (keyPressed == ord('q')):
        
        # print(hands)
        # print('--------------------')
        # print(face)
        # print('--------------------')
        # print(pose_lmList)
        # print('--------------------')
        # print(pose_bboxInfo)
        # sum = 0
        # if hands:
        #     sum += len(hands[0]['lmList'])
        #     if len(hands) == 2:
        #         sum += len(hands[1]['lmList'])
        # if (face):
        #     pass
        # if (pose_lmList):
        #     sum += len(pose_lmList)

        # print("lh: ", len(hands[0]['lmList']))
        # print("rh: ", len(hands[1]['lmList']) if len(hands) == 2 else "" )
        # print("pose: ", len(pose_lmList))
        # print("SUM: ", sum)
        pass

    if (keyPressed == 27):
        cam.release()
        cv2.destroyAllWindows()
        break


In [4]:
cam.release()
cv2.destroyAllWindows()

In [18]:
detectionResults['hands']

[{'lmList': [[193, 269, 0],
   [214, 285, -12],
   [240, 295, -19],
   [262, 301, -25],
   [279, 309, -31],
   [262, 260, -14],
   [291, 264, -24],
   [308, 268, -31],
   [322, 270, -37],
   [255, 245, -15],
   [281, 232, -23],
   [298, 225, -31],
   [313, 218, -37],
   [240, 235, -17],
   [253, 221, -27],
   [262, 212, -35],
   [272, 203, -40],
   [221, 231, -20],
   [223, 219, -30],
   [223, 211, -35],
   [224, 202, -38]],
  'bbox': (193, 202, 129, 107),
  'center': (257, 255),
  'type': 'Right'}]

3. Feature Extraction (Extract Landmark Values + Calculate Extra Features)

In [34]:
def featureExtraction(results):

    # Pose Landmarks
    pose_lm = np.array([each[1:] for each in results['pose_landmarks']]).flatten()

    if len(results['hands']) > 0:
        left_hand = None
        right_hand = None
        
        if results['hands'][0]['type'] == 'Left':
            left_hand = results['hands'][0]
            if (len(results['hands']) == 2):
                right_hand = results['hands'][1]
        else:
            right_hand = results['hands'][0]
            if (len(results['hands']) == 2):
                left_hand = results['hands'][1]


        # Left Hand Landmarks


    pass

featureExtraction(detectionResults)

[ 292  363 -425  309  332 -432  319  331 -432  329  329 -433  286  334
 -380  279  335 -380  272  336 -380  355  337 -293  283  345 -102  317
  379 -365  287  382 -317  465  459 -188  237  431   46  525  601 -317
   94  370  -22  558  717 -465  209  263 -139  577  756 -504  235  236
 -195  562  750 -537  249  242 -130  549  739 -485  244  255 -132  427
  811  -86  269  807   88  435 1096 -210  279 1082  -32  444 1362   29
  303 1353   53  451 1405   37  307 1390   50  423 1431 -333  309 1434
 -308]


In [40]:
detectionResults['hands'][0]

{'lmList': [[193, 269, 0],
  [214, 285, -12],
  [240, 295, -19],
  [262, 301, -25],
  [279, 309, -31],
  [262, 260, -14],
  [291, 264, -24],
  [308, 268, -31],
  [322, 270, -37],
  [255, 245, -15],
  [281, 232, -23],
  [298, 225, -31],
  [313, 218, -37],
  [240, 235, -17],
  [253, 221, -27],
  [262, 212, -35],
  [272, 203, -40],
  [221, 231, -20],
  [223, 219, -30],
  [223, 211, -35],
  [224, 202, -38]],
 'bbox': (193, 202, 129, 107),
 'center': (257, 255),
 'type': 'Right'}

4. Extract Extra Features

In [12]:
# Calculate the center 
def distance(pointsArr):
    # return np.linalg.norm(np.array(a) - np.array(b))
    pass

testData = np.random.random(size=(258 // 3, 1 * 3)) * 6 - 3

print(min(xLandmarks), max(xLandmarks))
print(np.min(testData), np.max(testData))
print(testData.shape)




-0.6764288544654846 2.961961507797241
-2.991369457452593 2.960440034291155
(86, 3)


In [26]:
print(xLandmarks[:9])
print(landmarks.pose_landmarks.landmark[:3])
import timeit
test = xLandmarks.reshape(258 // 3, 3)
print(test[:9])
print(test.shape)

[ 0.45312276  0.78516418 -0.46312538  0.99993253  0.48683429  0.73243636
 -0.42282629  0.99985081  0.50434029]
[x: 0.4531227648258209
y: 0.7851641774177551
z: -0.4631253778934479
visibility: 0.9999325275421143
, x: 0.4868342876434326
y: 0.7324363589286804
z: -0.42282629013061523
visibility: 0.9998508095741272
, x: 0.5043402910232544
y: 0.7309010624885559
z: -0.42261797189712524
visibility: 0.9998418092727661
]
[[ 0.45312276  0.78516418 -0.46312538]
 [ 0.99993253  0.48683429  0.73243636]
 [-0.42282629  0.99985081  0.50434029]
 [ 0.73090106 -0.42261797  0.99984181]
 [ 0.52128971  0.73023671 -0.4223803 ]
 [ 0.99985176  0.45039302  0.73809391]
 [-0.38673082  0.99984783  0.44226074]
 [ 0.73923188 -0.38610789  0.99982017]
 [ 0.43313536  0.74061579 -0.38619661]]
(86, 3)
